In [2]:
import pandas as pd
from poverty_income.csv_extractor import CSVExtractor
from poverty_income.api_extractor import APIExtractor
import json

In [3]:
with open("config.json") as config_file:
    config = json.load(config_file)
config

{'data_sources': {'csv': {'poverty': 'resources/allpovu.csv'},
  'api': {'income': 'https://apps.bea.gov/api/data/?UserID=41EB9172-D1E2-4C69-A09D-94AA8F280D3D&method=GetData&datasetname=Regional&TableName=CAINC1&LineCode=3&Year=2019&GeoFips=COUNTY&ResultFormat=json'},
  'sqlite3': {'host': ':memory:'}}}

In [4]:
poverty_df = CSVExtractor(config["data_sources"]["csv"]).extract()
poverty_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,July 2019 ACS-Like Poverty Universe for 2018 Estimates,Unnamed: 5,Unnamed: 6,Unnamed: 7,July 2018 ACS-Like Poverty Universe for 2018 Estimates,Unnamed: 9,...,Unnamed: 82,Unnamed: 83,July 2000 CPS-Like Poverty Universe for IY 1999 Estimates,Unnamed: 85,Unnamed: 86,Unnamed: 87,July 1999 CPS-Like Poverty Universe for IY 1998 Estimates,Unnamed: 89,Unnamed: 90,Unnamed: 91
0,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related",...,"Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"
1,0,0,United States,US,320118791,52324733,71633453,19002162,"319,184,033","52,529,919",...,"71,741,141","19,181,906",276207757,51642359,71684956,18968750,271059449,51060953,71338364,19382484
2,1,0,Alabama,AL,4781642,779632,1069755,286287,"4,763,811","781,913",...,"1,104,080","296,196",4368014,804291,1120718,293558,4348444,789510,1088427,295264
3,1,1,Autauga County,AL,55364,9572,12807,NaN,"55,073","9,677",...,"12,377",NaN,43711,9245,12507,NaN,43524,8856,12148,NaN
4,1,3,Baldwin County,AL,220556,34878,46757,NaN,"215,255","34,508",...,"34,503",NaN,139273,25048,34302,NaN,136585,24609,33859,NaN


In [139]:
# get only 2019 data
poverty_df_2019 = poverty_df.loc[:,"Unnamed: 0":"Unnamed: 7"]
new_header = poverty_df_2019.iloc[0]
poverty_df_2019 = poverty_df_2019[1:]
poverty_df_2019.columns = new_header
#suppress "County" from name column
poverty_df_2019.Name = poverty_df_2019.Name.apply(lambda x: x.replace("County","").strip())
# suppress total per state as it is a duplication
poverty_df_2019 = poverty_df_2019[poverty_df_2019["County FIPS code"] != "0"]
#add geofips column
poverty_df_2019["GeoFips"] = poverty_df_2019["State FIPS code"] + poverty_df_2019["County FIPS code"].str.zfill(3)
# drop state and country fips
poverty_df_2019.drop(["State FIPS code", "County FIPS code"], axis=1, inplace=True)
# transform columns to numeric
numeric_columns = poverty_df_2019.columns.drop(["Name", "State Postal Code"])
poverty_df_2019[numeric_columns] = poverty_df_2019[numeric_columns].apply(pd.to_numeric).astype("Int64")
# column order
poverty_df_2019 = poverty_df_2019[["GeoFips","Name","State Postal Code","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"]]
poverty_df_2019.head()


,GeoFips,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"
3,1001,Autauga,AL,55364,9572,12807,<NA>
4,1003,Baldwin,AL,220556,34878,46757,<NA>
5,1005,Barbour,AL,21801,3717,5003,<NA>
6,1007,Bibb,AL,20217,3210,4430,<NA>
7,1009,Blount,AL,57246,9676,13069,<NA>


In [108]:
income_df = APIExtractor(config["data_sources"]["api"]).extract()
income_df.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,CAINC1-3,01001,"Autauga, AL",2019,Dollars,0,"43,917",NaN
1,CAINC1-3,01003,"Baldwin, AL",2019,Dollars,0,"47,485",NaN
2,CAINC1-3,01005,"Barbour, AL",2019,Dollars,0,"35,763",NaN
3,CAINC1-3,01007,"Bibb, AL",2019,Dollars,0,"31,725",NaN
4,CAINC1-3,01009,"Blount, AL",2019,Dollars,0,"36,412",NaN


In [127]:
import copy
# keep only interesting columns
income_df_clean = copy.deepcopy(income_df[["GeoFips","DataValue"]])
# transform to numeric columns
income_df_clean["GeoFips"] = income_df_clean["GeoFips"].apply(pd.to_numeric)
income_df_clean["DataValue"] = income_df_clean["DataValue"].str.replace(",","")
income_df_clean["DataValue"] = income_df_clean["DataValue"].apply(lambda x: pd.to_numeric(x,errors="coerce")).astype("Int64")

income_df_clean.head()

,GeoFips,DataValue
0,1001,43917
1,1003,47485
2,1005,35763
3,1007,31725
4,1009,36412


In [140]:
pd.merge(poverty_df_2019,income_df_clean, on="GeoFips").head(10)

,GeoFips,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4",DataValue
0,1001,Autauga,AL,55364,9572,12807,<NA>,43917
1,1003,Baldwin,AL,220556,34878,46757,<NA>,47485
2,1005,Barbour,AL,21801,3717,5003,<NA>,35763
3,1007,Bibb,AL,20217,3210,4430,<NA>,31725
4,1009,Blount,AL,57246,9676,13069,<NA>,36412
5,1011,Bullock,AL,8453,1516,2070,<NA>,29080
6,1013,Butler,AL,19090,3120,4217,<NA>,37523
7,1015,Calhoun,AL,110652,17481,23930,<NA>,38394
8,1017,Chambers,AL,32779,4940,6772,<NA>,35900
9,1019,Cherokee,AL,25852,3684,4880,<NA>,36432
